# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686612


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:04,  2.38s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:42,  1.70s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:30,  1.26s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:21,  1.09it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.04it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  6.29it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  5.28it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.68it/s]

not-logged-in-68d3d82149e88f995533    0.000519
not-logged-in-e385081e7d0d82fc95d4    0.000351
jammiedodgers                         0.001183
CThomas                               0.002971
tanya_p                               0.000126
ElisabethB                            0.000135
EvalynGJ                              0.000299
Camelron                              0.000159
shocko61                              0.000171
not-logged-in-e0e2484e6da165ec8e8e    0.000127
theetick                              0.004018
sn346808                              0.004873
not-logged-in-116eef09a8f6e73ef4da    0.246586
Daniel.brooker2                       0.000312
HubertK                               0.000776
mikhaila                              0.000197
kevinlacaille                         0.000964
KevinKosa                             0.000135
not-logged-in-14d09f70263c1fe15cc3    0.000159
BigSean                               0.000155
Koffer                                0.000179
wilsony7209  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())